In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
import jax.numpy as jnp
import jax_smi
jax_smi.initialise_tracking()
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(("models/llama-3-70b-1.gguf", "models/llama-3-70b-2.gguf"),
                                         device_map="auto:mp=4")

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-70B")

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
], tokenize=False)

In [10]:
from micrlhf.sampling import sample
sample(llama, tokenizer, prompt, batch_size=1, max_seq_len=128, do_sample=True,
    #    fold=True
    #    use_jit=False
       )

  0%|          | 0/69 [00:00<?, ?it/s]

["<|begin_of_text|>System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context.\n\nUser: Who are you?\n\nAssistant: Nice to meet you! I'm an artificial intelligence assistant designed to provide helpful and accurate information to your questions. I'm here to assist you in a friendly and polite manner. If I'm unsure or don't have enough information to provide an accurate answer, I'll let you know and try to find more information or resources to help you. So"]